In [1]:
import model_lib
import numpy as np
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)
import os
import time
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [2]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    NUM_WORKERS = 1
    PIN_MEMORY = True
    DATA_ORDER = "cw_ins"
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    
    # only flips
    MEAN_PIXEL = np.array([0.485, 0.456, 0.406],dtype=np.float32).reshape(1,1,-1)
    STD_PIXEL = np.array([0.229, 0.224, 0.225],dtype=np.float32).reshape(1,1,-1)
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    WIDTH = 224
    HEIGHT = 224
    CROP_SIZE = 224
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [3]:
train_image_dir = "/media/data/nishanth/aravind/train2017/"
val_image_dir = "/media/data/nishanth/aravind/val2017/"
config = ProposalConfig()
model_dir = "./models/"
train_pickle = "/home/aravind/re/data/train_cwid.pickle"
val_pickle = "/home/aravind/re/data/val_cwid.pickle"

In [4]:
# with open(train_pickle,"rb") as train_ann:
#     train_cwid = pickle.load(train_ann)
with open(val_pickle,"rb") as val_ann:
    val_cwid = pickle.load(val_ann)

In [5]:
# train_loader = model_lib.get_loader(train_cwid,config,train_image_dir)
val_loader = model_lib.get_loader(val_cwid,config,val_image_dir)

In [ ]:
import torch
import torch.nn.functional as F
from PIL import Image
net = model_lib.SimpleHGModel()

pretrained_dict = torch.load(model_dir+"model_mask_vgg_0_0.pt")
net = model_lib.SimpleHGModel()
net_dict = net.state_dict()

pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in net_dict}
net_dict.update(pretrained_dict) 
net.load_state_dict(net_dict)

# net.vgg.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").vgg)
# net.classifier.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").classifier)
net = net.cuda()
with torch.no_grad():
    for i,data in enumerate(val_loader):
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = data
        print(config.CLASS_NAMES[np.argmax(batch_class_ids.numpy())])
        a,b,c= batch_images.numpy(), batch_impulses.numpy(),batch_gt_responses.numpy()
        a = np.moveaxis(a,1,-1)
        b = np.moveaxis(b,1,-1)
        c = np.moveaxis(c,1,-1)
        Image.fromarray(((a[0]*config.STD_PIXEL + config.MEAN_PIXEL)*255).astype(np.uint8),"RGB").show()
        Image.fromarray((b[0][:,:,0]*128).astype(np.uint8),"L").show()
#         Image.fromarray((c[0][:,:,0]*128).astype(np.uint8),"L").show()
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = batch_images.cuda(),batch_impulses.cuda(),batch_gt_responses.cuda(),batch_class_ids.cuda()
        pred_class,pred_mask = net([batch_images,batch_impulses])
#         print(pred_class)
        pred_class = F.softmax(pred_class,dim=-1).squeeze()
        maxs, indices = torch.topk(pred_class,5,-1)
        pred_mask = (F.sigmoid(pred_mask).clamp(0.7)*255).squeeze().cpu().numpy()
        Image.fromarray(pred_mask.astype(np.uint8),"L").show()
#         print(maxs.shape,indices.shape)
        for i in range(5):
            print(maxs[i],indices[i],config.CLASS_NAMES[int(indices[i])])
#         print(batch_class_ids)
#         print(indices)
#         print("gt_class:",config.CLASS_NAMES[int(batch_class_ids[0])])
#         print("pred_class: ",pred_class[int(batch_class_ids[0])])
#         print(config.CLASS_NAMES[int(indices[0])])
        input()

backpack
tensor(0.8691, device='cuda:0') tensor(25, device='cuda:0') backpack
tensor(1.00000e-02 *
       8.1114, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(1.00000e-02 *
       1.2517, device='cuda:0') tensor(76, device='cuda:0') vase
tensor(1.00000e-03 *
       6.5940, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-03 *
       4.5578, device='cuda:0') tensor(4, device='cuda:0') motorcycle

cake
tensor(0.1950, device='cuda:0') tensor(46, device='cuda:0') bowl
tensor(0.1846, device='cuda:0') tensor(56, device='cuda:0') cake
tensor(0.1558, device='cuda:0') tensor(55, device='cuda:0') donut
tensor(1.00000e-02 *
       8.1688, device='cuda:0') tensor(75, device='cuda:0') clock
tensor(1.00000e-02 *
       5.0437, device='cuda:0') tensor(33, device='cuda:0') sports ball

bottle
tensor(0.8965, device='cuda:0') tensor(40, device='cuda:0') bottle
tensor(1.00000e-02 *
       4.8872, device='cuda:0') tensor(41, device='cuda:0') wine glass
tensor(1.00000e-02 *


sink
tensor(0.5992, device='cuda:0') tensor(72, device='cuda:0') sink
tensor(1.00000e-02 *
       9.4450, device='cuda:0') tensor(61, device='cuda:0') dining table
tensor(1.00000e-02 *
       6.6696, device='cuda:0') tensor(3, device='cuda:0') car
tensor(1.00000e-02 *
       4.2268, device='cuda:0') tensor(9, device='cuda:0') boat
tensor(1.00000e-02 *
       3.2930, device='cuda:0') tensor(14, device='cuda:0') bench

person
tensor(0.1440, device='cuda:0') tensor(77, device='cuda:0') scissors
tensor(0.1329, device='cuda:0') tensor(26, device='cuda:0') umbrella
tensor(0.1062, device='cuda:0') tensor(31, device='cuda:0') skis
tensor(1.00000e-02 *
       9.9670, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       5.8555, device='cuda:0') tensor(35, device='cuda:0') baseball bat

donut
tensor(0.1253, device='cuda:0') tensor(76, device='cuda:0') vase
tensor(0.1091, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(1.00000e-02 *
       7.0123, device='cud